In [1]:
import os, json,pickle
import pandas as pd

path = 'data/'


for filename in os.listdir(path):
    
    with open(path+filename) as json_file:
        data = json.load(json_file)
    
    ct = 0
    for i in range(len(data["response"]["docs"])):
        element = data["response"]["docs"][i]
        if 'langMaterial' in element:
            ct+=1
    print (filename,ct, round(ct/len(data["response"]["docs"])*100,2),"%")

economics.json 5060 3.51 %
germanDemocraticRepublic.json 95455 81.46 %
genealogy.json 35295 80.6 %
slavery.json 0 0.0 %
maps.json 0 0.0 %
firstWorldWar.json 52566 91.56 %
frenchNapoleonI.json 0 0.0 %
notaries.json 7704 21.71 %
catholicism.json 0 0.0 %


In [2]:
path = 'data/'

aggregating_fields = ["langMaterial","unitTitle","titleProper","scopeContent","topic"]

dataset = [["id","langMaterial","unitTitle","titleProper","scopeContent","topic","filename"]]

for filename in os.listdir(path):
    
    with open(path+filename) as json_file:
        data = json.load(json_file)
    
    ct = 0
    for i in range(len(data["response"]["docs"])):
        element = data["response"]["docs"][i]
        
        line = [element["id"]]
        for field in aggregating_fields:
            if field in element:
                line.append(element[field])
            else:
                line.append(" ")
        line.append(filename)
        dataset.append(line)
print (len(dataset))

454807


In [3]:
from collections import Counter

selected_dataset = [dataset[0]]+ [x for x in dataset if len(x[4])>3]


In [4]:
column_names = selected_dataset.pop(0)

df = pd.DataFrame(selected_dataset[1:], columns=column_names)



In [5]:
langs = []

for index, row in df.iterrows():
            langs.append(row["langMaterial"])
from collections import Counter

Counter(langs).most_common()

[(' ', 62087),
 ('ger', 53316),
 ('fre', 7474),
 ('lav', 28),
 ('lav rus', 18),
 ('heb', 14),
 ('rus', 13),
 ('pol', 11),
 ('rus pol', 2),
 ('fre ger eng ita rus', 2),
 ('fre eng', 2),
 ('fre eng ger ita', 2),
 ('lav rus pol', 1)]

In [8]:
from langdetect import detect

langs = []

check = set()

for index, row in df.iterrows():
    if len(row["langMaterial"])==3:
        langs.append(row["langMaterial"])
        if row["langMaterial"] not in check:
            print(row["langMaterial"])
            check.add(row["langMaterial"])
        
    else:
        try:
            l = detect(row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"])
            row["langMaterial"] = l
            langs.append(row["langMaterial"])
        except Exception as e:
            print (e)
            continue
        
from collections import Counter

Counter(langs).most_common()



fre
ger
lav
rus
heb
pol


[('ger', 53316),
 ('fr', 47864),
 ('de', 10982),
 ('fre', 7474),
 ('fi', 2716),
 ('it', 313),
 ('pl', 140),
 ('sv', 42),
 ('lav', 28),
 ('lv', 21),
 ('en', 17),
 ('heb', 14),
 ('rus', 13),
 ('pol', 11),
 ('es', 10),
 ('ca', 9)]

In [9]:
with open('larger_selected_dataset.pickle', 'wb') as f:
        pickle.dump(df, f)